In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation, add, Input, Add
import numpy as np
import matplotlib.pyplot as plt
import zipfile


In [0]:
#tf.debugging.set_log_device_placement(True)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# **Implementation starts here**

In [0]:
!cp "/content/gdrive/My Drive/cell_images.zip" "/content/cell_images.zip"


In [0]:
!unzip -q "/content/cell_images.zip"

In [0]:
data_path = '/content/cell_images'

# **Task 3.1**

In [104]:
img_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2) 
img_height = 128
img_width = 128
batch_size = 64

train_generator = img_data_gen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True, 
    seed = 1, 
    color_mode = 'rgb',
    subset='training') 

val_generator = img_data_gen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True, 
    seed = 1, 
    color_mode = 'rgb',
    subset='validation') 


Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [0]:
cnn = Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(img_height, img_width ,3)), # inputting colored images
    MaxPooling2D(),
    BatchNormalization(), 
    Conv2D(64, 3, activation='relu'), 
    MaxPooling2D(),
    BatchNormalization(), 
    Conv2D(128, 3, activation='relu'), 
    MaxPooling2D(),
    BatchNormalization(), 
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(), 
    Dense(1)
])

In [0]:
cnn.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [108]:
epochs = 10
result_cnn = cnn.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/10
344/344 [==============================] - 34s 100ms/step - loss: 0.1683 - accuracy: 0.9368 - val_loss: 0.9397 - val_accuracy: 0.7000
Epoch 2/10
344/344 [==============================] - 34s 98ms/step - loss: 0.1403 - accuracy: 0.9484 - val_loss: 0.1874 - val_accuracy: 0.9340
Epoch 3/10
344/344 [==============================] - 34s 99ms/step - loss: 0.1226 - accuracy: 0.9548 - val_loss: 0.1905 - val_accuracy: 0.9386
Epoch 4/10
344/344 [==============================] - 34s 99ms/step - loss: 0.1289 - accuracy: 0.9523 - val_loss: 0.2061 - val_accuracy: 0.9370
Epoch 5/10
344/344 [==============================] - 34s 99ms/step - loss: 0.1074 - accuracy: 0.9606 - val_loss: 0.2414 - val_accuracy: 0.9201
Epoch 6/10
344/344 [==============================] - 34s 100ms/step - loss: 0.0922 - accuracy: 0.9649 - val_loss: 0.2305 - val_accuracy: 0.9326
Epoch 7/10
344/344 [==============================] - 34s 100ms/step - loss: 0.0794 - accuracy: 0.9703 - val_loss: 0.1729 - val_accura

In [147]:
print("Accuracy:", np.mean(result_cnn.history['accuracy']))
print("Loss:", np.mean(result_cnn.history['loss']))
print("Validation Accuracy:", np.mean(result_cnn.history['val_accuracy']))
print("Validation Loss:", np.mean(result_cnn.history['val_loss']))

Accuracy: 0.9623134970664978
Loss: 0.1017047829926014
Validation Accuracy: 0.9019258677959442
Validation Loss: 0.29274305105209353


# **Task 3.2**

In [161]:
img_data_gen_aug = ImageDataGenerator(rescale=1./255, 
                                  validation_split=0.2, 
                                  rotation_range = 180, 
                                  zoom_range = 0.2,
                                  horizontal_flip = True, 
                                  ) 

train_generator_aug = img_data_gen_aug.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True, 
    seed = 1,
    color_mode = 'rgb',
    subset='training') 


Found 22048 images belonging to 2 classes.


In [0]:
cnn_aug = Sequential([
    Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)), # inputting colored images
    MaxPooling2D(),
    BatchNormalization(), 
    Conv2D(32, 3, activation='relu'), 
    MaxPooling2D(),
    BatchNormalization(), 
    Conv2D(16, 3, activation='relu'), 
    MaxPooling2D(),
    BatchNormalization(), 
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(), 
    Dense(1)
])

In [0]:
cnn_aug.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [164]:
epochs = 10
result_cnn_aug = cnn_aug.fit_generator(
    train_generator_aug,
    steps_per_epoch = train_generator_aug.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/10
689/689 [==============================] - 108s 157ms/step - loss: 0.3092 - accuracy: 0.8750 - val_loss: 0.6905 - val_accuracy: 0.7254
Epoch 2/10
689/689 [==============================] - 109s 158ms/step - loss: 0.2020 - accuracy: 0.9282 - val_loss: 0.1808 - val_accuracy: 0.9349
Epoch 3/10
689/689 [==============================] - 109s 158ms/step - loss: 0.1771 - accuracy: 0.9404 - val_loss: 0.1618 - val_accuracy: 0.9455
Epoch 4/10
689/689 [==============================] - 109s 159ms/step - loss: 0.1623 - accuracy: 0.9459 - val_loss: 0.1838 - val_accuracy: 0.9223
Epoch 5/10
689/689 [==============================] - 110s 159ms/step - loss: 0.1571 - accuracy: 0.9476 - val_loss: 0.1758 - val_accuracy: 0.9323
Epoch 6/10
689/689 [==============================] - 110s 160ms/step - loss: 0.1527 - accuracy: 0.9492 - val_loss: 0.1748 - val_accuracy: 0.9334
Epoch 7/10
689/689 [==============================] - 110s 160ms/step - loss: 0.1490 - accuracy: 0.9494 - val_loss: 0.1785 -

In [165]:
print("Accuracy:", np.mean(result_cnn_aug.history['accuracy']))
print("Loss:", np.mean(result_cnn_aug.history['loss']))
print("Validation Accuracy:", np.mean(result_cnn_aug.history['val_accuracy']))
print("Validation Loss:", np.mean(result_cnn_aug.history['val_loss']))


Accuracy: 0.938738203048706
Loss: 0.17469519078731538
Validation Accuracy: 0.9131006956100464
Validation Loss: 0.23045493215322493



Transformations applied on training set include:
- rotation
- zoom
- horizontal flip

**We can see that the on an average, validation set accuracy has increased by 1.1%, and the validation set loss has decreased by 0.63.**



# **Task 3.3**

## **Network with Residual Connections** 

In [34]:
img_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2) 
img_height = 128
img_width = 128
batch_size = 32

train_generator = img_data_gen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True, 
    seed = 1, 
    color_mode = 'rgb',
    subset='training') 

val_generator = img_data_gen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True, 
    seed = 1, 
    color_mode = 'rgb',
    subset='validation') 

no_conv = 32

input_layer = Input(shape=(img_height,img_width, 3))
res_net = Conv2D(no_conv, 3, padding="same")(input_layer)

# number of filters in each unit iteration
no_conv_list = [32]*7
no_conv_list.extend([64]*7)
no_conv_list.extend([128]*7)
no_conv_list.extend([256]*7)

prev_no = 32

# unit iterations
for no_conv in no_conv_list:
  prev_res_net = res_net

  if prev_no != no_conv:
    prev_res_net = Conv2D(no_conv, 1, strides = (2,2), padding="same")(prev_res_net)
    res_net = MaxPooling2D(pool_size=(2,2))(res_net)

  res_net = BatchNormalization()(res_net)
  res_net = Activation("relu")(res_net)
  res_net = Conv2D(no_conv, 3, padding="same")(res_net)

  res_net = BatchNormalization()(res_net)
  res_net = Activation("relu")(res_net)
  res_net = Conv2D(no_conv, 3, padding="same")(res_net)

  res_net = BatchNormalization()(res_net)
  res_net = Activation("relu")(res_net)
  res_net = Conv2D(no_conv, 3, padding="same")(res_net)

  #res_net = MaxPooling2D(pool_size=(2, 2), padding='same')(res_net)

  """res_net = BatchNormalization()(res_net)
  res_net = Activation("relu")(res_net)
  res_net = Conv2D(no_conv, 3, strides = (1,1), padding="same")(res_net)"""

  res_net = add([prev_res_net, res_net]) # residual connection 

  prev_no = no_conv

res_net = Dropout(0.2)(res_net)
res_net = MaxPooling2D(pool_size=(3,3))(res_net)
res_net = Flatten()(res_net)
#res_net = Dense(128, activation="relu")(res_net)
res_net = Dense(1)(res_net)

res_net_model = Model(input_layer, res_net)

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [0]:
res_net_model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [36]:
res_net_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_364 (Conv2D)             (None, 128, 128, 32) 896         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_345 (BatchN (None, 128, 128, 32) 128         conv2d_364[0][0]                 
__________________________________________________________________________________________________
activation_345 (Activation)     (None, 128, 128, 32) 0           batch_normalization_345[0][0]    
____________________________________________________________________________________________

In [20]:
epochs = 5

result_res_net = res_net_model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/5
689/689 [==============================] - 544s 789ms/step - loss: 0.8325 - accuracy: 0.9157 - val_loss: 176.8365 - val_accuracy: 0.7729
Epoch 2/5
689/689 [==============================] - 543s 789ms/step - loss: 0.6359 - accuracy: 0.9253 - val_loss: 17.2988 - val_accuracy: 0.8390
Epoch 3/5
689/689 [==============================] - 543s 789ms/step - loss: 0.3760 - accuracy: 0.9450 - val_loss: 2.2505 - val_accuracy: 0.8330
Epoch 4/5
689/689 [==============================] - 544s 790ms/step - loss: 0.3787 - accuracy: 0.9304 - val_loss: 1.2409 - val_accuracy: 0.8884
Epoch 5/5
689/689 [==============================] - 543s 788ms/step - loss: 0.6393 - accuracy: 0.9037 - val_loss: 2.4728 - val_accuracy: 0.8003


In [21]:
print("Accuracy:", np.mean(result_res_net.history['accuracy']))
print("Loss:", np.mean(result_res_net.history['loss']))
print("Validation Accuracy:", np.mean(result_res_net.history['val_accuracy']))
print("Validation Loss:", np.mean(result_res_net.history['val_loss']))


Accuracy: 0.9240021705627441
Loss: 0.5724824845790863
Validation Accuracy: 0.8267441868782044
Validation Loss: 40.01987645626068


## **Network without Residual Connections** 

In [0]:
no_conv = 32

input_layer = Input(shape=(img_height,img_width, 3))
deep_net = Conv2D(no_conv, 3, padding="same")(input_layer)

# number of filters in each unit iteration
no_conv_list = [32]*7
no_conv_list.extend([64]*7)
no_conv_list.extend([128]*7)
no_conv_list.extend([256]*7)

prev_no = 32

# unit iterations
for no_conv in no_conv_list:
  prev_deep_net = deep_net

  if prev_no != no_conv:
    prev_deep_net = Conv2D(no_conv, 1, strides = (2,2), padding="same")(prev_deep_net)
    deep_net = MaxPooling2D(pool_size=(2,2))(deep_net)

  deep_net = BatchNormalization()(deep_net)
  deep_net = Activation("relu")(deep_net)
  deep_net = Conv2D(no_conv, 3, padding="same")(deep_net)

  deep_net = BatchNormalization()(deep_net)
  deep_net = Activation("relu")(deep_net)
  deep_net = Conv2D(no_conv, 3, padding="same")(deep_net)

  deep_net = BatchNormalization()(deep_net)
  deep_net = Activation("relu")(deep_net)
  deep_net = Conv2D(no_conv, 3, padding="same")(deep_net)

  #res_net = MaxPooling2D(pool_size=(2, 2), padding='same')(res_net)

  """res_net = BatchNormalization()(res_net)
  res_net = Activation("relu")(res_net)
  res_net = Conv2D(no_conv, 3, strides = (1,1), padding="same")(res_net)"""

  #deep_net = add([prev_res_net, deep_net]) # residual connection 

  prev_no = no_conv

deep_net = Dropout(0.2)(deep_net)
deep_net = MaxPooling2D(pool_size=(3,3))(deep_net)
deep_net = Flatten()(deep_net)
#res_net = Dense(128, activation="relu")(res_net)
deep_net = Dense(1)(deep_net)

deep_net_model = Model(input_layer, deep_net)

In [0]:
deep_net_model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [43]:
deep_net_model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_540 (Conv2D)          (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_513 (Bat (None, 128, 128, 32)      128       
_________________________________________________________________
activation_513 (Activation)  (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_541 (Conv2D)          (None, 128, 128, 32)      9248      
_________________________________________________________________
batch_normalization_514 (Bat (None, 128, 128, 32)      128       
_________________________________________________________________
activation_514 (Activation)  (None, 128, 128, 32)      0   

In [0]:
epochs = 5

result_deep_net = deep_net_model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/5
689/689 [==============================] - 512s 743ms/step - loss: 0.7080 - accuracy: 0.5659 - val_loss: 0.7419 - val_accuracy: 0.5002
Epoch 2/5
689/689 [==============================] - 512s 744ms/step - loss: 0.6445 - accuracy: 0.6104 - val_loss: 0.7402 - val_accuracy: 0.5979
Epoch 3/5
689/689 [==============================] - 513s 744ms/step - loss: 0.6178 - accuracy: 0.6394 - val_loss: 0.6984 - val_accuracy: 0.5200
Epoch 4/5
689/689 [==============================] - 513s 745ms/step - loss: 0.5710 - accuracy: 0.6926 - val_loss: 0.7242 - val_accuracy: 0.5065
Epoch 5/5
689/689 [==============================] - 512s 743ms/step - loss: 0.5061 - accuracy: 0.7519 - val_loss: 1.2902 - val_accuracy: 0.5004


## **Result Comparison (Network with residual connections and without residual connections)**

we can clearly see that where the average validation set accuracy of model with residual connections is around **82.6%**, the average validation set accuracy of the model with no residual connections is around **52%**.

This shows that the model with no residual connections is unable to train on the data given such deep network. 